# Lights!

### Setting up a kernel

In [3]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as skaoai
import os

import sys
sys.path.append('../') #needed to get util in the path

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

kernel = sk.Kernel()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

deployment="gpt-4o"
service_id = "default"
kernel.add_service(
    skaoai.AzureChatCompletion(
        deployment_name=deployment,
        endpoint=endpoint,api_key=api_key)
    )

print("You made a kernel!")

You made a kernel!


### Loading the functions in the kernel

In [4]:
plugins_directory = "../sk_plugins"
lights_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="lights_plugin")


#### Using Autoinvoke, so semantic kernel decides when to call a plugin by itself

In [5]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior

# Consider the following car status {{car_plugins.GetCarStatus}}

prompt = """
Perform the following actions:
{{$input}}
"""

execution_settings = kernel.get_service(service_id).instantiate_prompt_execution_settings(
    service_id=service_id, 
    temperature=0,  # Temperature can affect the execution of plugins!
    max_tokens=2000,
    function_call_behavior=FunctionCallBehavior.AutoInvokeKernelFunctions(),
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The actions to be performed", is_required=True),
    ],        
    execution_settings=execution_settings,
)

light_function = kernel.add_function(
    function_name="light_function",
    plugin_name="lights_plugin",
    prompt_template_config=prompt_template_config,
)


In [26]:
summary = await kernel.invoke(light_function, input="What is the status of the lights in the KITCHEN?")
print(summary)

The lights in the KITCHEN are currently OFF.


In [7]:
summary = await kernel.invoke(light_function, input="Switch on the the lights in the KITCHEN")
print(summary)


Changing light status for KITCHEN to on
Changing light status for KITCHEN to ON
The lights in the KITCHEN have been switched on.


In [9]:
summary = await kernel.invoke(light_function, input="Bring light to 50% to the KITCHEN")
print(summary)

Changing light status for KITCHEN to 50%
It seems that setting the light to 50% is not a valid option. The available statuses might be "on" or "off". Would you like to turn the light on or off in the kitchen?


In [29]:
summary = await kernel.invoke(light_function, input="What is the status of the lights in the KITCHEN?")
print(summary)

The lights in the KITCHEN are currently ON.


In [11]:
summary = await kernel.invoke(light_function, input="What is the status of the lights in the BEDROOM?")
print(summary)


The lights in the BEDROOM are currently OFF.


In [12]:
summary = await kernel.invoke(light_function, input="Which rooms are available?")
print(summary)

The available rooms are:

1. Living Room
2. Kitchen
3. Bedroom
4. Bathroom
5. Garage

Would you like to perform any actions related to these rooms?


In [13]:
summary = await kernel.invoke(light_function, input="What are the possible status for the lights")
print(summary)

The possible statuses for the lights are typically "on" and "off".
